In [9]:
import os
print(os.listdir('/content/'))


['.config', '.ipynb_checkpoints', 'movies_dataset.csv', 'sample_data']


In [11]:
import pandas as pd

df = pd.read_csv('/content/movies_dataset.csv')
print(df.head())


   MovieID                         Title        Genre  ReleaseYear  \
0        1          Might toward capital       Comedy         2003   
1        2         He however experience       Comedy         1988   
2        3  Star responsibility politics       Comedy         1971   
3        4                  Exactly live       Comedy         1998   
4        5      Focus improve especially  Documentary         2021   

  ReleaseDate Country   BudgetUSD  US_BoxOfficeUSD  Global_BoxOfficeUSD  \
0  28-09-2003   China  6577427.79       6613685.82          15472035.66   
1  14-02-1988     USA  1883810.10       1930949.15           3637731.12   
2  02-11-1971     USA  2468079.29       4186694.69           7165111.24   
3  06-08-1998     USA  1447311.46       2023683.92           4373820.26   
4  17-12-2021   India   900915.86       2129629.10           3113017.38   

   Opening_Day_SalesUSD  One_Week_SalesUSD  IMDbRating  RottenTomatoesScore  \
0            1778530.85         3034053.32       

In [16]:
import numpy as np
from sklearn.model_selection import StratifiedKFold
from sklearn.linear_model import LogisticRegression
from sklearn.impute import SimpleImputer  # Corrected import
from sklearn.metrics import accuracy_score
import pandas as pd

# Create binary target variable based on revenue median
# Assuming 'Global_BoxOfficeUSD' is the intended revenue column
df['revenue_class'] = np.where(df['Global_BoxOfficeUSD'] > df['Global_BoxOfficeUSD'].median(), 1, 0)

# Select features for classification
# Updated feature names to match the DataFrame columns
features = ['BudgetUSD', 'IMDbRating', 'RottenTomatoesScore', 'NumVotesIMDb', 'NumVotesRT']

# Prepare features and handle missing values
X = df[features].copy() # Use .copy() to avoid SettingWithCopyWarning
imputer = SimpleImputer(strategy='mean')
X_imputed = imputer.fit_transform(X)

# Target variable
y = df['revenue_class'].values

# Stratified 5-fold cross-validation setup
kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

model = LogisticRegression(max_iter=200)
accuracies = []

# Train and evaluate using k-fold
for fold, (train_idx, test_idx) in enumerate(kf.split(X_imputed, y), 1):
    X_train, X_test = X_imputed[train_idx], X_imputed[test_idx] # Corrected line
    y_train, y_test = y[train_idx], y[test_idx]

    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    acc = accuracy_score(y_test, y_pred)
    accuracies.append(acc)

    print(f"Fold {fold} Accuracy: {acc:.4f}")

print(f"\nAverage Accuracy: {np.mean(accuracies):.4f}")

Fold 1 Accuracy: 0.8741
Fold 2 Accuracy: 0.8731
Fold 3 Accuracy: 0.8677
Fold 4 Accuracy: 0.8750
Fold 5 Accuracy: 0.8673

Average Accuracy: 0.8714
